## Bonusový úkol č. 2 - stahování dat z webového zdroje
Vytvořte funkci **sync()**, která získá kompletní seznam produktů (tj. včetně dalších stránek) dostupných v kategorii
https://www.alza.cz/bezzrcadlovky-bez-objektivu/18863907.htm
a u každého produktu zjistí jeho aktuální cenu a stav skladu.
Funkce bude uchovávat získané informace a historii změn v relační databázi SQLLite3 obsahující dvě tabulky:  
* tabulku `products` a  
* tabulku `products_history`.

Struktura obou tabulek je shodná a obsahuje následující sloupce:  
* `id` TEXT - id produktu, např. OS072i1l1 (viz data-impression-id),  
* `url` TEXT - url produktu k kterému se vztahuje cena (pouze část path, viz ukázka na konci),  
* `title` TEXT - název produktu,  
* `price` DECIMAL - cena produktu s DPH k danému datu,   
* `stock_state` TEXT - stav skladu k danému datu,  
* `last_update` DATETIME - datum poslední změny hodnot záznamu v UTC  

Do tabulky `products_history` zkopírujte záznam z tabulky `products` ve chvíli, kdy se změnil nějaký sledovaný údaj (název, cena nebo stav skladu) a je potřeba aktualizovat data v tabulce `products`. Pozor, jedno `id` může mít více variant `url` s různou cenou. Při opětovném volání funkce **sync()** se prověří existence záznamu v `products`, prověří se shoda hodnot a vždy aktualizuje hodnota `last_update`, aby bylo zřejmé, ke kterému datu je informace platná.

**Předpokládaná náročnost**: 1 hodina

### Závislosti, načtení knihoven

V následující buňce deklarujte všechny závislosti

In [2]:
#%pip install requests requests_cache bs4

import requests, requests_cache, sqlite3, random, json
from datetime import datetime
from bs4 import BeautifulSoup

#pro vývoj je vhodné zapnout cache (viz přednáška), pro finalní otestovaní tento řádek zakomentujte
#requests_cache.install_cache('devel') 

#nadeklarujeme si novy typ sloupce DECIMAL do sqlite3, abychom měli automatický převod mezi SQLite3 a Python
from decimal import Decimal
sqlite3.register_adapter(Decimal, lambda d: str(d))
sqlite3.register_converter("DECIMAL", lambda s: Decimal(s.decode('ascii')))

### Deklarace funkce

V následujícím boxu definujte funkci **sync(name)** s jedním parametrem (název souboru s DB), která provede zadanou operaci. 
Pro přístup k DB lze s ohledem na složitost zadání použít přímo funkcionalitu vestavěného modulu sqlite3 (viz https://docs.python.org/2/library/sqlite3.html).

**TIP**: pro získání seznamu všech produktů lze použít endpoint https://www.alza.cz/Services/EShopService.svc/Filter

Mohlo by se také hodit: https://curl.trillworks.com/

In [3]:
# V tomto boxu pouze implementujte funkci ale nevolejte ji (pro vývoj si vytvořte vlastní buňky).
# nezapomeňte na cookies a hlavičky, jinak se Vám může zobrazit otázka "nejste robot?"
def sync(dbfile='data.sqlite'):
    with sqlite3.connect(dbfile, detect_types=sqlite3.PARSE_DECLTYPES) as conn:
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS products
                  (id TEXT, url TEXT, title TEXT, price DECIMAL, stock_state TEXT, last_update DATETIME, PRIMARY KEY(id,url))''')
        
        c.execute('''CREATE TABLE IF NOT EXISTS products_history
                  (id TEXT, url TEXT, title TEXT, price DECIMAL, stock_state TEXT, last_update DATETIME)''')

        c.execute('''CREATE INDEX IF NOT EXISTS idx_id ON products (id)''')
        c.execute('''CREATE INDEX IF NOT EXISTS idx_idurl ON products_history (id, url)''')

        s = requests.session()

        headers = {
            'authority': 'www.alza.cz',
            'accept-language': 'cs-CZ',
            'accept': 'application/json, text/javascript, */*; q=0.01',
            'cache-control': 'no-cache',
            'x-requested-with': 'XMLHttpRequest',
            'request-id': '|973a07ce13a44edba8ecab0829c730ac.2eb748de1b504297',
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36',
            'content-type': 'application/json; charset=UTF-8',
            'origin': 'https://www.alza.cz',
            'sec-fetch-site': 'same-origin',
            'sec-fetch-mode': 'cors',
            'sec-fetch-dest': 'empty',
            'referer': 'https://www.alza.cz/bezzrcadlovky-bez-objektivu/18863907.htm',
            'cookie': '__uzma=269bbfd1-409b-4fef-b122-51cd3ee787e9; __uzmb=1602658052; __uzme=1427; VZTX=2735082486; lb_id=ec39c75ec53053efa06a93196f63c29d; TPL=1; CCC=18863907; CriticalCSS=6858194; .AspNetCore.Culture=c%3Dcs-CZ%7Cuic%3Dcs-CZ; ai_user=VFIEOX9/evJhQ7Ph1KGuOf|2020-10-14T11:52:53.916Z; _vwo_uuid_v2=D08B456C3BBB48BD311C83C833E6C5C33|d4930ecd58f346b0706be6ee2ba75ec8; __ssds=2; __ssuzjsr2=a9be0cd8e; __uzmaj2=70bf9707-aafa-42da-ac0f-0d2f2a4dd313; __uzmbj2=1602676388; i18next=cs-CZ; _gid=GA1.2.1921946261.1602676375; _gcl_au=1.1.926748260.1602676377; _fbp=fb.1.1602676379135.650136970; SL_C_23361dd035530_VID=f8pLsnWpLbH; SL_C_23361dd035530_KEY=b0375d591e85b0affec1d581bfed3c760cb8c56d; db_ui=3174fc5a-1b9e-2b10-731c-25406f61d5e3; _hjTLDTest=1; _hjid=d638cd1e-04b5-4619-b8af-3cc44e04d07c; _hjIncludedInSessionSample=1; _hjAbsoluteSessionInProgress=0; db_uicd=bf422192-1455-2de6-f4e1-a7c98c312677; PVCFLP=1; __uzmcj2=788552875524; __uzmdj2=1602678709; SL_C_23361dd035530_SID=0lsvqeY--Fe; _ga_FGLGFS7LP0=GS1.1.1602676376.1.1.1602678728.26; hvrcomm=5786787; ai_session=r7LhRxUH60EZU+MpNrffPU|1602676374467|1602678765141; _ga=GA1.2.48707104.1602676375; _gat_UA-948269-48=1; __uzmc=683386747974; __uzmd=1602678806; sc/bezzrcadlovky-bez-objektivu/18863907.htm=15281',
        }

        data = '{"idCategory":18863907,"producers":"","parameters":[],"idPrefix":0,"prefixType":0,"page":1,"pageTo":4,"inStock":false,"newsOnly":false,"commodityStatusType":null,"upperDescriptionStatus":0,"branchId":-2,"sort":0,"categoryType":1,"searchTerm":"","sendProducers":false,"layout":0,"append":false,"leasingCatId":null,"yearFrom":null,"yearTo":null,"artistId":null,"minPrice":-1,"maxPrice":-1,"shouldDisplayVirtooal":false,"callFromParametrizationDialog":false,"commodityWearType":null,"scroll":15281,"hash":"#f&cst=null&cud=0&pg=1-4&prod=","counter":3}'

        response = requests.post('https://www.alza.cz/Services/EShopService.svc/Filter', headers=headers, data=data)


        response_dict = response.json()
        app_json = json.dumps(response_dict)
        json_object = json.dumps(response_dict, indent = 4)   
        json2 = response_dict['d']
        json3 = json2['Boxes']
        json4 = BeautifulSoup(json3, "html.parser")
        products = json4.find_all("div", "browsingitem")
        
        for product in products:
            idd = product.find("a")['data-impression-id']
            url = product.find("a", {"class":"name browsinglink"})['href']
            title = product.find("a", {"class":"name browsinglink"}).get_text()
            if product.find("span", "c2") is not None:
                price = product.find("span", "c2").get_text()
                #price = price[:-2]
                price=''.join(i for i in price if i.isdigit())
            else:
                price = 0
                
            if product.find("a")['data-impression-dimension13'] is not None:
                stock_state = product.find("a")['data-impression-dimension13']
            else:
                stock_state = 0
        
           
            time = datetime.now()
        
            newRecord = None
            c.execute('''SELECT *
                               FROM products
                               WHERE id == ? AND url == ?''',(idd, url))
            newRecord = c.fetchone()

            if newRecord is None:
                    c.execute('''REPLACE INTO products (id, url, title, price, stock_state, last_update) VALUES(?,?,?,?,?,?)''',(idd, url, title, price, stock_state,time))
            else:                              
                if (str(price) != str(newRecord[3])) or (stock_state != newRecord[4]) or (title != newRecord[2]):          
                    c.execute('''DELETE FROM products_history WHERE id == ? AND url == ?''', (idd, url))
                    c.execute('''REPLACE INTO products_history SELECT id,url, title, price, stock_state, last_update FROM products WHERE id = ? AND url = ?''',(idd, url))                    
                    c.execute('''REPLACE INTO products (id, url, title, price, stock_state, last_update) VALUES(?,?,?,?,?,?)''',(idd, url, title, price, stock_state,time))
       
        conn.commit()  
        c.close()

### Ověření korektní funkce

Na následujícím kódu lze ověřit základní funkcionalitu. Měly byste dostat stejný výstup jako je v ukázce. Protože se však stav e-shopu může měnit, uzpůsobte si eventuelně dotaz dle potřeb. Momentálně se testuje existence produktu https://www.alza.cz/sony-alpha-7ii?dq=2286288 ev. 
https://www.alza.cz/kod/OS072i1p5.

Při ověřování korektní funkce Vaší implementace bude porovnán obsah DB vytvořený Vaší funkcí s předpokládaným obsahem DB v určitou dobu a poté znovu s několika hodinovým odstupem.

In [4]:
from contextlib import closing

sync('data.sqlite')

with sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_DECLTYPES) as conn:
    with closing(conn.cursor()) as c:
        c.execute('SELECT id, url, price FROM products WHERE id=? AND url=? AND price>20000', ('OS072i1p5','/sony-alpha-7ii?dq=2286288'))
        r = c.fetchone()
        print(r)
        assert(r != None)

        c.execute('SELECT id, url, price FROM products WHERE id=? AND price>30000', ('OF7032a',))
        r = c.fetchall()
        print(r)
        assert (len(r)>0 and '/fujifilm-x-t3?dq=5457426' in [a[1] for a in r])

print("OK")        

('OS072i1p5', '/sony-alpha-7ii?dq=2286288', Decimal('31490'))
[('OF7032a', '/fujifilm-x-t3?dq=5457426', Decimal('39990')), ('OF7032a', '/fujifilm-x-t3-telo-cerny-levne-d5754350.htm', Decimal('36990')), ('OF7032a', '/fujifilm-x-t3-telo-cerny-sleva-d5877920.htm', Decimal('33990'))]
OK


### Komentář
Do pole níže můžete vložit textový komentář týkající se tohoto úkolu. Např. jak dlouho Vám trvalo řešení, co bylo obtížné, co bylo se mělo více v rámci přenášky vysvětlit apod.

n/a